In [0]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D,Flatten
from keras.models import Model
from keras import backend as K

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Activation, Reshape, UpSampling2D,Dropout
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot


import numpy as np
import pandas as pd
import io


In [0]:
seen_data_training = pd.read_csv('seen-dataset/seen-dataset/dataset_seen_training_siamese.csv')
seen_data_validation= pd.read_csv('dataset_seen_validation_siamese.csv')

shuffle_data_training =pd.read_csv('unseen-dataset/unseen-dataset/dataset_seen_training_siamese.csv')
shuffle_data_validation= pd.read_csv('dataset_Shseen_validation_siamese.csv')

unseen_data_training = pd.read_csv('dataset_unseen_training_siamese.csv')
unseen_data_validation= pd.read_csv('dataset_unseen_validation_siamese.csv')


seen_data_training=seen_data_training.drop('Unnamed: 0',axis=1)
shuffle_data_training=shuffle_data_training.drop('Unnamed: 0',axis=1)
unseen_training=unseen_data_training.drop('Unnamed: 0',axis=1)


seen_data_validation=seen_data_validation.drop('Unnamed: 0',axis=1)
shuffle_data_validation=shuffle_data_validation.drop('Unnamed: 0',axis=1)
unseen_data_validation=unseen_data_validation.drop('Unnamed: 0',axis=1)

In [0]:
l_train=seen_data_training['left']
r_train=seen_data_training['right']
target_seen=seen_data_training['label'].values.tolist()



l_train_shuffle=shuffle_data_training['left']
r_train_shuffle=shuffle_data_training['right']
target_seen_shuffle=shuffle_data_training['label'].values.tolist()


l_train_un=unseen_data_training['left']
r_train_un=unseen_data_training['right']
target_seen_un=unseen_data_training['label'].values.tolist()

l_train1=seen_data_validation['left']
r_train1=seen_data_validation['right']
target_seen1=seen_data_validation['label'].values.tolist()


l_train1_shuffle=shuffle_data_validation['left']
r_train1_shuffle=shuffle_data_validation['right']
target_seen1_shuffle=shuffle_data_validation['label'].values.tolist()

l_train1_un=unseen_data_validation['left']
r_train1_un=unseen_data_validation['right']
target_seen1_un=unseen_data_validation['label'].values.tolist()


In [0]:
def train_data_proc(path,loop_l,loop_r,target):
  import PIL
  from scipy import misc
  import imageio
  count = 0
  X_train=[]
  for name in loop_l:
#     print(count)
    try:
      X_train.append(np.array(PIL.Image.open(path+"/"+name).convert("L")))
      count += 1
    except:
  #     del target_seen[count]
        pass
  
  print("something done")
  count=0
  X_train1=[]
  for name in loop_r:
  #   print(count)
      try:
        X_train1.append(np.array(PIL.Image.open(path+"/"+name).convert("L")))
        count += 1
      except Exception as e:
        
      # Remove count index in target_seen
        del target[count]
      # Remove count index in left_target
        del X_train[count]
        
  
  return X_train,X_train1

 
    

In [0]:
X_train,X_train1=train_data_proc("seen-dataset/seen-dataset/TrainingSet",l_train,r_train,target_seen)


In [0]:
def valid_data_proc(path,loop_l,loop_r,target):
  import PIL
  from scipy import misc
  import imageio
  X_train_v=[]
  for name in loop_l:
    X_train_v.append(np.array(PIL.Image.open(path+"/"+name).convert("L")))
  
  count=0
  X_train1_v=[]
  for name in loop_r:
  #   print(count)
    try:
      X_train1_v.append(np.array(PIL.Image.open(path+"/"+name).convert("L")))
      count += 1
    except Exception as e:
      # Remove count index in target_seen
      del target[count]
    # Remove count index in left_target
      del X_train_v[count]
      
  return X_train_v , X_train1_v 



In [0]:
X_train,X_train1=train_data_proc("seen-dataset/seen-dataset/TrainingSet",l_train,r_train,target_seen)
#print(X_train)
X_train_sh,X_train1_sh=train_data_proc("shuffled-dataset/shuffled-dataset/TrainingSet",l_train_shuffle,r_train_shuffle,target_seen_shuffle)
X_train_un,X_train1_un=train_data_proc("unseen-dataset/unseen-dataset/TrainingSet",l_train_un,r_train_un,target_seen_un)

In [ ]:
X_train_v , X_train1_v=valid_data_proc("seen-dataset/seen-dataset/ValidationSet",l_train1,r_train1,target_seen1)
X_train_v_sh , X_train1_v_sh=valid_data_proc("shuffled-dataset/shuffled-dataset/ValidationSet",l_train1_shuffle,r_train1_shuffle,target_seen1_shuffle)
X_train_v_un , X_train1_v_un=valid_data_proc("unseen-dataset/unseen-dataset/ValidationSet",l_train1_un,r_train1_un,target_seen1_un)

In [0]:
X_train=np.array(X_train)
X_train1=np.array(X_train1)
Y_train=np.array(target_seen)


X_train_un=np.array(X_train_un)
X_train1_un=np.array(X_train1_un)
Y_train_un=np.array(target_seen_un)

X_train_shuffle=np.array(X_train_sh)
X_train1_shuffle=np.array(X_train1_sh)
Y_train_shuffle=np.array(target_seen_shuffle)

X_train_v=np.array(X_train_v)
X_train1_v=np.array(X_train1_v)
Y_train1=np.array(target_seen1)


X_train_v_un=np.array(X_train_v_un)
X_train1_v_un=np.array(X_train1_v_un)
Y_train1_un=np.array(target_seen1_un)

X_train_v_shuffle=np.array(X_train_v_sh)
X_train1_v_shuffle=np.array(X_train1_v_sh)
Y_train1_shuffle=np.array(target_seen1_shuffle)

In [0]:
input_img = Input(shape=(64, 64, 1)) 

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)
encoded = Flatten()(x)
encoded = Dense(8*8*8, activation='relu', name='latent')(encoded)

r = Reshape(target_shape=(8,8,8))(encoded)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(r)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same',name='output')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [0]:
encoder = Model(autoencoder.inputs, autoencoder.get_layer('latent').output)

In [0]:
autoencoder.fit(X_train.reshape(len(X_train),64,64,1), X_train1.reshape(len(X_train1),64,64,1),
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(X_train_v.reshape(len(X_train_v),64,64,1), X_train1_v.reshape(len(X_train1_v),64,64,1)),
               )

In [0]:
preds=autoencoder.predict((X_train_v.reshape(len(X_train_v),64,64,1)))

In [0]:

import os

display(csv_data.head()), display(csv_data.shape)
train_path = 'seen-dataset/seen-dataset/TrainingSet/'
train_images = os.listdir(train_path)

val_path = 'seen-dataset/seen-dataset/ValidationSet/'
val_images = os.listdir(val_path)
len(train_images)+len(val_images)

In [0]:
dataset_columns = csv_data.columns
csv_data['imagename'] = csv_data['imagename'].astype('str')
display(csv_data.head(20))
csv_data.shape
f = []
for col in dataset_columns[1:]:
    f.append(csv_data[col].unique().shape[0])
f,len(f)

In [0]:
feature_lengths = {}
for col in dataset_columns[1:]:
    feature_lengths[col] = len(csv_data[col].unique())
feature_lengths = list(feature_lengths.values())
feature_lengths

In [0]:
input_layer = Input((64,64,1))
model = BatchNormalization()(input_layer)

model = Conv2D(activation='relu',filters=32, kernel_size=(32,32), padding='same')(model)
model = Conv2D(activation='relu',filters=64, kernel_size=(32,32), strides=(1,1), padding='same')(model)
model = MaxPooling2D((2,2))(model)
model = Dropout(0.25)(model)

model = Conv2D(activation='relu',filters=128, kernel_size=(16,16), padding='same')(model)
model = Conv2D(activation='relu',filters=256, kernel_size=(16,16), strides=(1,1), padding='same')(model)
model = MaxPooling2D((2,2))(model)
model = Dropout(0.25)(model)

model = Conv2D(activation='relu',filters=128, kernel_size=(8,8), padding='same')(model)
model = Conv2D(activation='relu',filters=256, kernel_size=(8,8), strides=(1,1), padding='same')(model)
model = MaxPooling2D((2,2))(model)
model = Dropout(0.25)(model)


dense_layer = Dense(512, activation='relu')(model)
dense_layer = Dropout(0.5)(dense_layer)
out_dense_layer=[]
for i in range(1,len(f)+1):
    out_dense_layer.append(Dense(f[i-1] , activation='softmax', name = 'out_feature_'+str(i))(Flatten()(Dense(64 , activation='relu', name = 'dense_layer_'+str(i))(dense_layer))))

human_features = Model(inputs=[input_layer], outputs=out_dense_layer)
human_features.summary()

In [0]:
losses = {}
lossWeights = {}
for i in range(1,16):
    losses["out_feature_"+str(i)] = "categorical_crossentropy"
    lossWeights["out_feature_"+str(i)] = 1.0

In [ ]:
mc = ModelCheckpoint(filepath='15_human_features_weights_unseen_v1.h5', monitor='val_loss',period=1,save_best_only=True,save_weights_only=True,mode='auto',verbose=3)
es = EarlyStopping(patience=10000, monitor='val_loss', min_delta=0.0005, mode='auto')

In [0]:
from keras.optimizers import Adadelta, SGD, Adam
EPOCHS = 10
INIT_LR = 0.0001
opt = Adadelta(lr=INIT_LR, decay=1e-04)
human_features.compile(optimizer=opt, loss=losses, loss_weights=lossWeights,metrics=["accuracy"])
human_features.summary()

In [0]:
tg = (n for n in list(preds))
vg=(n for n in list(X_train_v))

In [0]:

hist = human_features.fit(preds,epochs=EPOCHS
                                    , steps_per_epoch = 1
                                    , validation_data = X_train_v
                                    , validation_steps = 1
                                    , verbose = 3
                                    
                                   )

In [ ]:
df_hist = pd.DataFrame(hist.history)
df_hist.plot(subplots=True, figsize=(15,100))
print(df_hist['val_loss'].min())
df_hist[df_hist['val_loss']<9.75]

In [0]:

# human_features.load_weights('human_features_weights_v2.h5')
human_features.load_weights('15_human_features_weights_unseen_v1.h5')
vx,vo=next(vg)
# print(len(vo))
fig = plt.figure(figsize=(80,200))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=1.51, wspace=0.05)
for i in range(64):
#     print(i, '\nPredicted:')
    
    real=''
    for f in range(15):
#         print(vo[i][f])
        real+=str(np.argmax(vo[f][i])+1)+','
    
    f_probs = human_features.predict(np.expand_dims(vx[i],0))
#     print(f_probs)
    pred=''
    for prob in f_probs:
#         print(prob)
        pred+=str(np.argmax(prob[0])+1)+','
    
    ax = fig.add_subplot(64, 1, i + 1, xticks=[], yticks=[])
    ax.imshow(vx[i].reshape((64,64)))
    ax.set_title('real:'+real+'\n pred:'+pred)
#     print('\n----------')
#     print('True:')
#     for op in o:
#         print(np.argmax(op[i]),end=',')
        
#     print('\n++++++',i,'++++++')

In [ ]:
fig = plt.figure(figsize=(30,30))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
for i in range(30):
    ax = fig.add_subplot(1, 30, i + 1, xticks=[], yticks=[])
    ax.imshow(inputs[64-30:64][i].reshape((64,64)))
    ax.set_title(i+34)

In [ ]:
plt.imshow(inputs[10].reshape((64,64)))
plt.subplots_adjust(hspace = 0.1)
for i in range(1,9):
    visualize_model = Model(human_features.input, human_features.get_layer('dense_layer_'+str(i)).output)
    vis_out = visualize_model.predict(np.expand_dims(inputs[10],0))[0]
    vis_out = np.rollaxis(a=vis_out, start=0, axis=2)
    f,ax = plt.subplots(8,8)
    f.subplots_adjust(hspace = .00, wspace=.02, top=0.95)
    f.set_size_inches(15,15)
    f.suptitle('dense_layer_'+str(i), fontsize=15)
#     f.subplots_adjust(hspace = 0.1)
#     plt.title('dense_layer_'+str(i))
    print(vis_out.shape)
    for r in range(8):
        for c in range (8):
            ax[r,c].imshow(vis_out[c+r*8])
            ax[r,c].axis('off')
#     break

In [ ]:
csv_data.var()

In [ ]:
test_g = datagen(len(X_train_v)*5,vshift=(-10,10), list_of_writers=X_train_v)
test_x,test_y = next(test_g)

In [ ]:
pred_y=human_features_eval.predict(test_x)

In [ ]:
pred_y=np.array(pred_y)

In [ ]:
pred_y[:,0,:].shape

In [ ]:

test_im_normal = test_x[0]#255.0-cv2.imread('../Handwriting/cleaned_AND_data_64x64/0301c_num2.png',0)
test_im_shifted = test_x[1]#255.0-cv2.imread('../Handwriting/cleaned_AND_data_64x64/0301c_num1.png',0)
f,ax = plt.subplots(1,2)
f.subplots_adjust(hspace = .00, wspace=.05)
f.set_size_inches(5,1*3)
ax[0].imshow(test_im_normal.reshape((64,64)))
ax[0].axis('off')

ax[1].imshow(test_im_shifted.reshape((64,64)))
ax[1].axis('off')
ax[0].set_title('Left Input')
ax[1].set_title('Right Input')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook

l = pred_y[:,0,:]
r = pred_y[:,1,:]

features = csv_data.keys()[1:]
feature_similarity_score=cosine_similarity(l,r).diagonal()
df_feat_sim_score = pd.DataFrame(columns=['features','feature_similarity_score'])
df_feat_sim_score['features']=features
df_feat_sim_score['feature_similarity_score']=feature_similarity_score
df_feat_sim_score.plot.bar(x='features')
df_feat_sim_score['feature_similarity_score'].mean()

In [ ]:
class_preds = human_features.predict(test_x[:2])for i in range(len(test_x[:2])):
    for f in range(15):
        print(np.argmax(class_preds[f][i]),end=',')
    print('')

In [ ]:
csv_data.var()